In [1]:
import pandas as pd
import requests
from json import JSONDecodeError
from tqdm import tqdm
from time import sleep

In [2]:
# Read data
parkings = pd.read_csv('parkings.csv')
suppliers = pd.read_csv('suppliers.csv')
vehicles = pd.read_csv('vehicles.csv')

In [3]:
suppliers

,id,CompanyName,Address,phone,representative,email,description,sector,ProductAndService,Market,Masothue,Year,Scale,Capacity,Price
0,company_1,Công Ty TNHH MTV Minh Tâm Phát,"261 KP. 3, QL. 1A, TT. Bến Lức, H. Bến Lức, Lo...",0913 958 373,Mr. Phạm Hoàng Phát,minhtamphat@gmail.com,Công Ty TNHH MTV Minh Tâm Phát thành lập và ho...,Xay Xát - Nhà Máy Xay Xát Lúa Gạo,Xay Xát - Nhà Máy Xay Xát Lúa Gạo;Gạo - Công T...,Toàn quốc,1100523365,2004,Từ 11 - 50 người,121.031905,1849.033815
1,company_2,Công Ty Cổ Phần Sản Xuất Thương Mại Lúa Vàng Việt,"785-787 Cách Mạng Tháng tám, Khu Phố 2, Phường...",0945402710,Mr. Đào Duy Băng Thanh,thanhdao@ducthanhco.vn,Công Ty Lúa Vàng Việt là đơn vị sản xuất và cu...,Gạo - Công Ty Và Đại Lý Gạo,Gạo - Công Ty Và Đại Lý Gạo;Xay Xát - Nhà Máy ...,"HACCP, VSATTP",3901259282,2018,Từ 51 - 100 người,86.609079,2195.570942
2,company_3,Công Ty TNHH Chế Biến Nông Sản Song Long,"Thôn 3, Bắc Ruộng, Tánh Linh, Bình Thuận",0971 173 168,Nguyễn Thị La Ngà,langariver@gmail.com,Công Ty TNHH Chế Biến Nông Sản Song Long chuyê...,Xay Xát - Nhà Máy Xay Xát Lúa Gạo,Xay Xát - Nhà Máy Xay Xát Lúa Gạo;Công ty Dịch...,"Toàn quốc, Quốc tế",2017,Công Ty TNHH Chế Biến Nông Sản Song Long,Công ty Dịch vụ,63.281514,679.341685
3,company_4,Công ty TNHH Ngọc Ngọc Hà,"Số 15, Tổ 1, Ấp An Long, Xã An Thạnh Trung, Hu...",0972 272 244,Mr. Ngô Văn Nhanh,beriphong@Gmail.com,"Công ty chúng tôi chuyên sản xuất, xay xát lúa...",Xay Xát - Nhà Máy Xay Xát Lúa Gạo,Xay Xát - Nhà Máy Xay Xát Lúa Gạo;Sản xuất;Toà...,Toàn quốc,1601979776,2017,Từ 11 - 50 người,146.377439,1429.721761
4,company_5,Công Ty Cổ Phần Hoàng Minh Nhật,"Số 18/2, Đường tỉnh lộ 922, ấp Thới Khánh A, x...",No phone,Hotline,hmn@hmnfoodco.com,Công Ty CP Hoàng Minh Nhật đến nay đã có hơn 1...,Xay Xát - Nhà Máy Xay Xát Lúa Gạo,Xay Xát - Nhà Máy Xay Xát Lúa Gạo;Gạo - Công T...,HACCP,HACCP,Công Ty Cổ Phần Hoàng Minh Nhật,"Nhà sản xuất, Công ty Xuất Khẩu",61.603753,2296.384706
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111,company_112,Phước Sinh - Công Ty TNHH Thương Mại Dịch Vụ P...,"Khu Công Nghiệp Vĩnh Lộc, Lot C12/1,Đường 2F, ...",No phone,No representative,No email,No description,Xay Xát - Nhà Máy Xay Xát Lúa Gạo,"Xay Xát - Nhà Máy Xay Xát Lúa Gạo;Bột Mì, Bột ...","Bột Mì, Bột Ngô, Năng, Gạo, Nếp",No tax code,No year,No scale,79.262889,1563.263337
112,company_113,Nhà Máy Xát Xay Gạo Dung Chiếm,"Số 108, TDP. 2, Miêu Nha, Tây Mỗ, Q. Nam Từ Li...",No phone,No representative,No email,Nhà Máy Xát Xay Gạo Dung Chiếm chuyên cung cấp...,Gạo - Công Ty Và Đại Lý Gạo,Gạo - Công Ty Và Đại Lý Gạo;Xay Xát - Nhà Máy ...,ISO 9001:2000,2005,ISO 9001:2000,Nhà Máy Xát Xay Gạo Dung Chiếm,114.638510,2125.961370
113,company_114,Doanh Nghiệp Tư Nhân Tài Lợi,"153 ấp Xẻo Chích, TT. Châu Hưng, H. Vĩnh Lợi, ...",No phone,No representative,No email,Doanh nghiệp tư nhân Tài Lợi chuyên cung cấp g...,Gạo - Công Ty Và Đại Lý Gạo,Gạo - Công Ty Và Đại Lý Gạo;Xay Xát - Nhà Máy ...,Xay Xát - Nhà Máy Xay Xát Lúa Gạo,ít hơn 5 người,Doanh Nghiệp Tư Nhân Tài Lợi,1900173102,52.565777,2200.345896
114,company_115,Doanh Nghiệp Tư Nhân Bảo Vân,"63 Tổ 2, ấp Long Hòa, X. Long An, H. Tân Châu,...",No phone,No representative,No email,No description,Gạo - Công Ty Và Đại Lý Gạo,Gạo - Công Ty Và Đại Lý Gạo;Xay Xát - Nhà Máy ...,Xay Xát - Nhà Máy Xay Xát Lúa Gạo,No tax code,No year,No scale,188.667723,1734.662390


In [ ]:
#Test
output_format = 'json'
url = f'https://maps.gomaps.pro/maps/api/geocode/json'

API_KEY = ''
params = {
    'key': API_KEY,
    'address' : 'Võ Thị Sáu, P. Đông Thành, Ninh Bình',
    'language':'vi',
    'region':'vi'
}

def extract_location_data(api_response, addr):
    if api_response['status'] != 'OK':
        return None  # handle any error responses here
    
    result = api_response['results'][0]  # get the first result
    location_data = {
        "formatted_address": result.get("formatted_address"),
        "latitude": result['geometry']['location']['lat'],
        "longitude": result['geometry']['location']['lng'],
        "place_id": result.get("place_id"),
        "Address_key": addr
    }
    return location_data

addresses = []

for addr in tqdm(suppliers['Address'].to_list()):
    try:
        params['address'] = addr
        response = requests.get(url, params=params, timeout=5)
        rqjson = response.json()
        location_data = extract_location_data(rqjson, addr)
        if location_data in addresses:
            continue
        addresses.append(location_data)
    except JSONDecodeError as e:
        print(addr)
        print(response.json())
        sleep(3)
        params['address'] = addr
        response = requests.get(url, params=params, timeout=5)
        rqjson = response.json()
        location_data = extract_location_data(rqjson, addr)
        if location_data in addresses:
            continue
        addresses.append(location_data)
           

100%|████████████████████████████████████████████████████████████████████████████████| 116/116 [01:20<00:00,  1.44it/s]


In [7]:
addr_df=pd.DataFrame(addresses) 
addr_df

,formatted_address,latitude,longitude,place_id,Address_key
0,"3 QL1A, TT. Bến Lức, Bến Lức, Long An, Việt Nam",10.638192,106.478286,ChIJuf7OLWXMCjERcZpjTG4k9d0,"261 KP. 3, QL. 1A, TT. Bến Lức, H. Bến Lức, Lo..."
1,"785 Cách Mạng Tháng Tám, Hiệp Ninh, Tây Ninh, ...",11.308933,106.114195,ChIJGXPpnFNqCzERwiHs1hwPApw,"785-787 Cách Mạng Tháng tám, Khu Phố 2, Phường..."
2,"thôn 3, Tánh Linh, Bình Thuận, Việt Nam",10.919007,107.673745,ChIJaTn9vhw2dDEREnnE9GeBVOA,"Thôn 3, Bắc Ruộng, Tánh Linh, Bình Thuận"
3,"An Thạnh Trung, Tp. Long Xuyên, An Giang, Việt...",10.427419,105.488828,ChIJHZfm2O5tCjERxQMVEViFEac,"Số 15, Tổ 1, Ấp An Long, Xã An Thạnh Trung, Hu..."
4,"2 ĐT922, Định Môn, Thới Lai, Cần Thơ, Việt Nam",10.088376,105.590790,ChIJvQBCrsuaoDERycYt6NvSI0I,"Số 18/2, Đường tỉnh lộ 922, ấp Thới Khánh A, x..."
...,...,...,...,...,...
81,"RHHV+JHG Kcn, Đ. Vĩnh Lộc Đường Số 8, Vĩnh Lộc...",10.829054,106.593962,ChIJx6U7ILArdTERTFXuU3EkLtE,"Khu Công Nghiệp Vĩnh Lộc, Lot C12/1,Đường 2F, ..."
82,"2 P. Miêu Nha, Tây Mỗ, Nam Từ Liêm, Hà Nội, Vi...",21.011448,105.746647,ChIJI1WbVXtTNDERoCzASwPcCuA,"Số 108, TDP. 2, Miêu Nha, Tây Mỗ, Q. Nam Từ Li..."
83,"Châu Hưng, Tp. Bạc Liêu, Bạc Liêu, Việt Nam",9.338861,105.729249,ChIJGe4og18IoTERJAywwDzGQXU,"153 ấp Xẻo Chích, TT. Châu Hưng, H. Vĩnh Lợi, ..."
84,"63 Đường Lý Thái Tổ, TT. Tân Châu, Tân Châu, A...",10.797003,105.249833,Eko2MyDEkMaw4budbmcgTMO9IFRow6FpIFThu5UsIFRULi...,"63 Tổ 2, ấp Long Hòa, X. Long An, H. Tân Châu,..."


In [8]:
addr_df = pd.merge(suppliers,addr_df, left_on='Address', right_on='Address_key').drop(columns=['Address_key'])
addr_df

,id,CompanyName,Address,phone,representative,email,description,sector,ProductAndService,Market,Masothue,Year,Scale,Capacity,Price,formatted_address,latitude,longitude,place_id
0,company_1,Công Ty TNHH MTV Minh Tâm Phát,"261 KP. 3, QL. 1A, TT. Bến Lức, H. Bến Lức, Lo...",0913 958 373,Mr. Phạm Hoàng Phát,minhtamphat@gmail.com,Công Ty TNHH MTV Minh Tâm Phát thành lập và ho...,Xay Xát - Nhà Máy Xay Xát Lúa Gạo,Xay Xát - Nhà Máy Xay Xát Lúa Gạo;Gạo - Công T...,Toàn quốc,1100523365,2004,Từ 11 - 50 người,121.031905,1849.033815,"3 QL1A, TT. Bến Lức, Bến Lức, Long An, Việt Nam",10.638192,106.478286,ChIJuf7OLWXMCjERcZpjTG4k9d0
1,company_2,Công Ty Cổ Phần Sản Xuất Thương Mại Lúa Vàng Việt,"785-787 Cách Mạng Tháng tám, Khu Phố 2, Phường...",0945402710,Mr. Đào Duy Băng Thanh,thanhdao@ducthanhco.vn,Công Ty Lúa Vàng Việt là đơn vị sản xuất và cu...,Gạo - Công Ty Và Đại Lý Gạo,Gạo - Công Ty Và Đại Lý Gạo;Xay Xát - Nhà Máy ...,"HACCP, VSATTP",3901259282,2018,Từ 51 - 100 người,86.609079,2195.570942,"785 Cách Mạng Tháng Tám, Hiệp Ninh, Tây Ninh, ...",11.308933,106.114195,ChIJGXPpnFNqCzERwiHs1hwPApw
2,company_3,Công Ty TNHH Chế Biến Nông Sản Song Long,"Thôn 3, Bắc Ruộng, Tánh Linh, Bình Thuận",0971 173 168,Nguyễn Thị La Ngà,langariver@gmail.com,Công Ty TNHH Chế Biến Nông Sản Song Long chuyê...,Xay Xát - Nhà Máy Xay Xát Lúa Gạo,Xay Xát - Nhà Máy Xay Xát Lúa Gạo;Công ty Dịch...,"Toàn quốc, Quốc tế",2017,Công Ty TNHH Chế Biến Nông Sản Song Long,Công ty Dịch vụ,63.281514,679.341685,"thôn 3, Tánh Linh, Bình Thuận, Việt Nam",10.919007,107.673745,ChIJaTn9vhw2dDEREnnE9GeBVOA
3,company_4,Công ty TNHH Ngọc Ngọc Hà,"Số 15, Tổ 1, Ấp An Long, Xã An Thạnh Trung, Hu...",0972 272 244,Mr. Ngô Văn Nhanh,beriphong@Gmail.com,"Công ty chúng tôi chuyên sản xuất, xay xát lúa...",Xay Xát - Nhà Máy Xay Xát Lúa Gạo,Xay Xát - Nhà Máy Xay Xát Lúa Gạo;Sản xuất;Toà...,Toàn quốc,1601979776,2017,Từ 11 - 50 người,146.377439,1429.721761,"An Thạnh Trung, Tp. Long Xuyên, An Giang, Việt...",10.427419,105.488828,ChIJHZfm2O5tCjERxQMVEViFEac
4,company_5,Công Ty Cổ Phần Hoàng Minh Nhật,"Số 18/2, Đường tỉnh lộ 922, ấp Thới Khánh A, x...",No phone,Hotline,hmn@hmnfoodco.com,Công Ty CP Hoàng Minh Nhật đến nay đã có hơn 1...,Xay Xát - Nhà Máy Xay Xát Lúa Gạo,Xay Xát - Nhà Máy Xay Xát Lúa Gạo;Gạo - Công T...,HACCP,HACCP,Công Ty Cổ Phần Hoàng Minh Nhật,"Nhà sản xuất, Công ty Xuất Khẩu",61.603753,2296.384706,"2 ĐT922, Định Môn, Thới Lai, Cần Thơ, Việt Nam",10.088376,105.590790,ChIJvQBCrsuaoDERycYt6NvSI0I
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111,company_112,Phước Sinh - Công Ty TNHH Thương Mại Dịch Vụ P...,"Khu Công Nghiệp Vĩnh Lộc, Lot C12/1,Đường 2F, ...",No phone,No representative,No email,No description,Xay Xát - Nhà Máy Xay Xát Lúa Gạo,"Xay Xát - Nhà Máy Xay Xát Lúa Gạo;Bột Mì, Bột ...","Bột Mì, Bột Ngô, Năng, Gạo, Nếp",No tax code,No year,No scale,79.262889,1563.263337,"RHHV+JHG Kcn, Đ. Vĩnh Lộc Đường Số 8, Vĩnh Lộc...",10.829054,106.593962,ChIJx6U7ILArdTERTFXuU3EkLtE
112,company_113,Nhà Máy Xát Xay Gạo Dung Chiếm,"Số 108, TDP. 2, Miêu Nha, Tây Mỗ, Q. Nam Từ Li...",No phone,No representative,No email,Nhà Máy Xát Xay Gạo Dung Chiếm chuyên cung cấp...,Gạo - Công Ty Và Đại Lý Gạo,Gạo - Công Ty Và Đại Lý Gạo;Xay Xát - Nhà Máy ...,ISO 9001:2000,2005,ISO 9001:2000,Nhà Máy Xát Xay Gạo Dung Chiếm,114.638510,2125.961370,"2 P. Miêu Nha, Tây Mỗ, Nam Từ Liêm, Hà Nội, Vi...",21.011448,105.746647,ChIJI1WbVXtTNDERoCzASwPcCuA
113,company_114,Doanh Nghiệp Tư Nhân Tài Lợi,"153 ấp Xẻo Chích, TT. Châu Hưng, H. Vĩnh Lợi, ...",No phone,No representative,No email,Doanh nghiệp tư nhân Tài Lợi chuyên cung cấp g...,Gạo - Công Ty Và Đại Lý Gạo,Gạo - Công Ty Và Đại Lý Gạo;Xay Xát - Nhà Máy ...,Xay Xát - Nhà Máy Xay Xát Lúa Gạo,ít hơn 5 người,Doanh Nghiệp Tư Nhân Tài Lợi,1900173102,52.565777,2200.345896,"Châu Hưng, Tp. Bạc Liêu, Bạc Liêu, Việt Nam",9.338861,105.729249,ChIJGe4og18IoTERJAywwDzGQXU
114,company_115,Doanh Nghiệp Tư Nhân Bảo Vân,"63 Tổ 2, ấp Long Hòa, X. Long An, H

In [9]:
addr_df.to_csv('suppliers_v2.csv', encoding='utf8', index=False)

In [ ]:
output_format = 'json'
url = f'https://maps.gomaps.pro/maps/api/geocode/json'

API_KEY = ' '
params = {
    'key': API_KEY,
    'address' : 'Võ Thị Sáu, P. Đông Thành, Ninh Bình',
    'language':'vi',
    'region':'vi'
}

def extract_location_data(api_response, addr):
    if api_response['status'] != 'OK':
        return None  # handle any error responses here
    
    result = api_response['results'][0]  # get the first result
    location_data = {
        "formatted_address": result.get("formatted_address"),
        "latitude": result['geometry']['location']['lat'],
        "longitude": result['geometry']['location']['lng'],
        "place_id": result.get("place_id"),
        "Address_key": addr
    }
    return location_data

for addr in tqdm(parkings['address'].to_list()):
    try:
        params['address'] = addr
        response = requests.get(url, params=params, timeout=5)
        rqjson = response.json()
        location_data = extract_location_data(rqjson, addr)
        if location_data in addresses:
            continue
        addresses.append(location_data)
    except JSONDecodeError as e:
        print(addr)
        print(response.json())
        sleep(3)
        params['address'] = addr
        response = requests.get(url, params=params, timeout=5)
        rqjson = response.json()
        location_data = extract_location_data(rqjson, addr)
        if location_data in addresses:
            continue
        addresses.append(location_data)

addr_df=pd.DataFrame(addresses) 
addr_df = pd.merge(parkings,addr_df, left_on='address', right_on='Address_key').drop(columns=['Address_key'])
addr_df.to_csv('parkings_v2.csv', encoding='utf8', index=False)
addr_df

100%|██████████████████████████████████████████████████████████████████████████████████| 18/18 [00:16<00:00,  1.11it/s]


,Unnamed: 0,id,name,address,formatted_address,latitude,longitude,place_id
0,0,parking_1,Bãi Đỗ Xe VSIP 1,"Đường số 2, Khu Công Nghiệp VSIP I, Thủ Dầu Mộ...","8 Hữu Nghị, Binh Hoà, Thuận An, Bình Dương, Vi...",10.924386,106.713579,ChIJn9_WQW_XdDERIn6MOQBROdc
1,1,parking_2,Bãi Đỗ Xe Tân Bình,"246 Đường D1, Khu Công Nghiệp VSIP I, Thủ Dầu ...","8 Hữu Nghị, Binh Hoà, Thuận An, Bình Dương, Vi...",10.924386,106.713579,ChIJn9_WQW_XdDERIn6MOQBROdc
2,2,parking_3,Bãi Đỗ Xe Mỹ Phước,"Khu Công Nghiệp Mỹ Phước 1, Bến Cát, Bình Dương","Số 85 Đường D18A, KCN, Bến Cát, Bình Dương, Vi...",11.117166,106.600089,ChIJ3605rg7NdDERHHGECUzcZS0
3,3,parking_4,Bãi Đỗ Xe Mỹ Phước 2,"Khu Công Nghiệp Mỹ Phước 2, Bến Cát, Bình Dương","4JJJ+R73, NA 6, Mỹ Phước, Bến Cát, Bình Dương,...",11.132015,106.630669,ChIJyyuAnKjOdDERcOCBNl1Ajv8
4,4,parking_5,Bãi Đỗ Xe Trảng Bàng,"Đường Nguyễn Thái Học, Trảng Bàng, Tây Ninh","Trảng Bàng, Tây Ninh, Việt Nam",11.030504,106.359266,ChIJAb8nBZsvCzERplsXjFn1808
5,5,parking_6,Bãi Đỗ Xe Lê Lợi,"100 Đường Lê Lợi, Trảng Bàng, Tây Ninh","100 Lê Hồng Phong, Trảng Bàng, Tây Ninh, Việt Nam",11.034381,106.359998,EjcxMDAgTMOqIEjhu5NuZyBQaG9uZywgVHLhuqNuZyBCw6...
6,6,parking_7,Bãi Đỗ Xe Ngã Ba Vũng Tàu,"Quốc lộ 51, Ngã Ba Vũng Tàu, Long Thành, Đồng Nai","QL51, Long Thành, Đồng Nai, Việt Nam",10.749723,106.969930,EidRTDUxLCBMb25nIFRow6BuaCwgxJDhu5NuZyBOYWksIF...
7,7,parking_8,Bãi Đỗ Xe Thế Giới Di Động,"64 Quốc lộ 51, Long Thành, Đồng Nai","64 QL51, Tam Phước, Biên Hòa, Đồng Nai, Việt Nam",10.854000,106.928117,Ejc2NCBRTDUxLCBUYW0gUGjGsOG7m2MsIEJpw6puIEjDsm...
8,8,parking_9,Bãi Đỗ Xe Cần Thơ,"Đường Trần Văn Khéo, P. An Khánh, Q. Ninh Kiều...","Trần Văn Khéo, Cái Khế, Ninh Kiều, Cần Thơ, Vi...",10.045925,105.786444,ChIJgfFsIKlioDERkVFEnJNNBKg
9,9,parking_10,Bãi Đỗ Xe Chợ Nổi Cái Răng,"Khu vực chợ nổi, Cái Răng, TP. Cần Thơ","An Bình, Ninh Kiều, Cần Thơ, Việt Nam",10.005036,105.745982,ChIJjS00tq-JoDERmDjx9Wm1ss0


In [ ]:
API_KEY = ' '
headers = {
    'X-Goog-FieldMask':'routes.distanceMeters,routes.duration',
    'Content-Type':'application/json',
    'X-Goog-Api-Key':API_KEY
}
url = 'https://routes.gomaps.pro/directions/v2:computeRoutes'
data = {
    "origin":{
        "address":"Nguyễn Thị Định, Tp. Bến Tre, Bến Tre, Việt Nam"
    },
    "destination":{
        "address":"An Bình, Ninh Kiều, Cần Thơ, Việt Nam"
    },
    "travelMode":'drive',
    "routingPreference": "ROUTING_PREFERENCE_UNSPECIFIED",
    "polylineQuality": "high_quality",
    "computeAlternativeRoutes": False,
    "routeModifiers": {
        "avoidTolls": False,
        "avoidHighways": False,
        "avoidFerries": False,
        "avoidIndoor": False
    }
}

response = requests.post(url, headers=headers, json=data)
response.json()

{'routes': [{'legs': [{'distanceMeters': 124453,
     'duration': '7842s',
     'staticDuration': '7842s',
     'polyline': {'encodedPolyline': 'sxq}@exjhSS`@[n@Sd@Q`@aA|BkAjCUh@y@nBaAxBUb@k@xAk@nAk@pAYr@CHq@vA[j@}BvFMb@G`@Gf@CXADGn@Gd@CZ?`@?RDf@yCg@gAC_@E}DoAuDk@w@MsFy@wDiAcAOcAOs@Mo@KcAQ_AMgAOqCa@GAw@MyAUoBYmCa@_G}@OCUvAAHK~@YfCGh@K`AEVIv@MhAQ|AQbB]|CCXG`@?@QdBIr@QzAADGb@In@I\\I\\Mb@ITEHGVM`@U~@Uv@YdAOf@kAfEIZg@fBKZOn@K\\GRSx@]jAg@hBA?Qt@Qj@ELU|@aAfD_@rAa@xAU|@CH{AlFyAlFUz@s@fCW~@g@jBOl@GZGVOhAMhAMhAEd@K`ACPGn@]xCE\\MjAAHId@QdAEVCh@GlA?@Iv@CR]zCIz@Gh@KdAADI|@In@CXs@~Gq@|FK`AEd@E\\C\\F\\mDl@eAPkARs@JuARgC\\gAd@mGzA_B`@cAVa@HeAV{@Tw@RSFYFa@Je@Ja@HwBXmGz@OBiBPwEj@uGt@c@Da@Fe@FiCZA?a@DA@oDb@}BXK@E@E@IF}Db@cEf@qD`@sD`@]Bu@BWBWDQ@E@qFr@sAN_CXcALq@H{Eh@E@E?kAL}@Hu@F}BReAJA?uFd@yALmAJuE`@G@mCViBNmCV_BLmCRC?M?i@?AAA??AAAA?A??AA?A?AAA?A?A?A?A??@A?A?A?A@A@A?A@A@A@e@RcCVoE^{ALa@D}CTkE`@c@BsAL{@J[BeFf@k@HgCXmANcBTyB\\k@JE@A@CDiANa@FwDh@oBXu@JMA[BcBV{@LaC\\_@DcALUHUF{@JsC\\kAN]HuFp@a@FuAPcEf@O@OB{

In [111]:
response.json().get('routes')

[{'legs': [{'distanceMeters': 1590433,
    'duration': '92196s',
    'staticDuration': '92196s',
    'polyline': {'encodedPolyline': 'aatzBo|yeSZHb@Fb@DPBpAHXCP@B_@Fi@Je@FULc@ZeAzAd@bBn@lC|@hDjAx@XtAf@fBn@~@\\fAb@dA^dA`@fA`@fA^pAb@fBj@dEtA`@LDDDFJDj@TFBZJ`@FTFZLNFHBHBJDHBJFLDN@F@^TXNr@^hAh@|@h@xAx@j@VJD^N`@N^PhIfDp@RlA^`Bf@j@R|@XzA`@dF`Bd@Px@ZNHdDhBvChA|Al@rBn@VHJB|A^VDp@XPBbANdANh@H^Fh@HbC`@dFr@zANF@|@FPBr@F~CXx@Hx@Hh@DpAHL@|AJ~ALzAJR?\\?n@Kf@Kf@I\\G\\Il@Hr@NB@|A^JHdAZ^N^P\\?DAD?J@d@@d@BhAF`BJx@Fh@Bp@Jt@Jb@H~Ab@fA`@B@fAh@pAp@h@T`ExBXPb@Xp@`@h@Z\\Np@Zn@T^J|A`@`AVn@L\\TNJXNJH`@XFxC@d@TdJBfAEjCEBCBCBCBCFADADAD?D?D@D@D@D@DBBBDBBDBB@@@B?D@D@D?B?DADADABCDCBCBCBG@GbB]j@El@EtDYpFa@lHc@fGc@dAGhDOZAzACF?F?|CB@?P?bAARAH?XC@@@@@??@@?@@@?@?@??@@?@A@?@?@A@?@??A@??A@?n@BH@jBLvFAfIBLArH^pCTlE^B?\\BtEn@B?lBV\\F^DB@rAPv@HjAJnCVTDB?bBT`Fp@x@TVDn@NPJp@^~AjA@?XXfC`C|@v@x@n@lA|@BBbAp@r@`@fAn@nAt@h@Zp@`@|@d@h@XnBdAZPXLt@\\x@b@RNn@d@H`@NVPNn@~CTjAZ`ARn@Lb@Vx@DJPd@NVTR@FNXd@~@?@JTLTtBhElBtDbApBBDNXBD?@NVVf@BF

In [123]:
def print_structure(data, indent=0):
    prefix = " " * indent
    if isinstance(data, dict):
        for key, value in data.items():
            print(f"{prefix}{key}:")
            print_structure(value, indent + 4)
    elif isinstance(data, list) and data:
        print(f"{prefix}- List of {len(data)} items:")
        print_structure(data[0], indent + 4)  # Assuming list items have the same structure
    else:
        print(f"{prefix}{type(data).__name__}")

print_structure(response.json().get('routes')[0])

legs:
    - List of 1 items:
        distanceMeters:
            int
        duration:
            str
        staticDuration:
            str
        polyline:
            encodedPolyline:
                str
        startLocation:
            latLng:
                latitude:
                    float
                longitude:
                    float
        endLocation:
            latLng:
                latitude:
                    float
                longitude:
                    float
        steps:
            - List of 17 items:
                distanceMeters:
                    int
                staticDuration:
                    str
                polyline:
                    encodedPolyline:
                        str
                startLocation:
                    latLng:
                        latitude:
                            float
                        longitude:
                            float
                endLocation:
                    l

In [124]:
received = response.json().get('routes')[0].get('localizedValues')
received

{'distance': {'text': '124 km'},
 'duration': {'text': '2 giờ 11 phút'},
 'staticDuration': {'text': '2 giờ 11 phút'}}

In [128]:
import regex as re 
hours =received['duration']['text']
# x ngày y giờ z phút t giây => k giờ (float)
def convert_to_hours(time_str):
    pattern = r"(?:(\d+)\s*ngày)?\s*(?:(\d+)\s*giờ)?\s*(?:(\d+)\s*phút)?\s*(?:(\d+)\s*giây)?"
    match = re.match(pattern, time_str)
    
    days = int(match.group(1)) if match.group(1) else 0
    hours = int(match.group(2)) if match.group(2) else 0
    minutes = int(match.group(3)) if match.group(3) else 0
    seconds = int(match.group(4)) if match.group(4) else 0

    total_hours = days * 24 + hours + minutes / 60 + seconds / 3600
    return round(total_hours, 2)
hours =received['duration']['text']    
hours = convert_to_hours(hours)

distance = received['distance']['text']
distance = distance.strip().split(' ')[0]
distance = re.sub(f'[.,]', '', distance)

print(hours)
print(distance)

2.18
124


In [98]:
str(addr_df['address'][0])

'Đường số 2, Khu Công Nghiệp VSIP I, Thủ Dầu Một, Bình Dương'

In [99]:
# Travel matrix

In [ ]:
route_data = [ # for each possible routes
    {
        "route_id": 1,
        "supplier_id": "S1",
        "Supplier_Address":"",
        "destination_address":""
        "weight": 20,  # in tons, weight this route can deliver
        **"travel_hours": 5,  # total driving time in hours
        "driver_cost": 100,  # salary cost for this route = Number of Routes Taken * 2 * 24000
        **"fuel_cost": 50,     # fuel cost for this route = total KM * 21427.5 * 100/3.3
        "goods_cost": 300,   # cost of goods for this route (price per ton * weight)
        "total_cost": 450,   # sum of driver_cost, fuel_cost, and goods_cost
    },

In [ ]:
'''
input: dataframe of suppliers (name, id, address), parking_areas(name, id, address), destination(str address)

Construct a route selection algorithm:
- Pair all suppliers with destination
- Send requests to get travel hours, travel kilometers
- For each pair that has travel hours > 8, fragment them by adding the closest parking area to the middle long-lat coordinate of the route to the route, breaking it to supp-parking1 and parking1-dest. 
- Recursively implement the above so that for a pair of dest-parking, parking-parking, parking-dest, sup-dest, driving hour do not exceed 8 hours
- The final result may looks like:
[
{
"route_id":'id',
"vehicle": plate num,
"weight": 20,  # in tons, weight this route can deliver
**"travel_hours": 5,  # total driving time in hours
"driver_cost": 100,  # salary cost for this route = Number of Routes Taken * 2 * 24000
**"fuel_cost": 50,     # fuel cost for this route = total KM * 21427.5 * 100/3.3
"goods_cost": 300,   # cost of goods for this route (price per ton * weight)
"total_cost": 450,   # sum of driver_cost, fuel_cost, and goods_cost
"routes": [[sup, parking1], [parking1, dest]]},
{...,
"routes": [[sup, dest]],
},
{
...,
"routes": [[sup, parking1], [parking1, parking2], [parking2, dest]]
},
...
]
'''

In [152]:
headers = {
    'Content-Type':'application/json',

    }
API_KEY = 'AlzaSy8750ykJid8J1QxUZxMFWO0j9MUwcFxSPR'
url = 'https://maps.googleapis.com/maps/api/geocode/json'
data={
        'address': 'Nguyễn Thị Định, Tp. Bến Tre, Bến Tre, Việt Nam',
        'key':API_KEY,
    }
response = requests.post(url, headers=headers, params=data)
response.json()

{'error_message': 'The provided API key is invalid. ',
 'results': [],
 'status': 'REQUEST_DENIED'}